In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

from datasets import CustomCIFAR10 as CIFAR10_dataset
from victim_model import *
from utils import *

from consts import *

from torch.utils.tensorboard import SummaryWriter
import pickle

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize(cifar10_mean, cifar10_std)
    ])
batch_size = 32

trainset= CIFAR10_dataset("../data/", transform=transform, train = True,download=True)
testset= CIFAR10_dataset("../data/", transform=transform, train = False,download=True)

trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=1, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
torch.hub.set_dir('/mnt/DONNEES/hbrachemi/.cache/torch/hub/checkpoints/')

In [4]:
model_ft= VictimModel("vgg16",True,10)

/mnt/DONNEES/hbrachemi/anaconda/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/DONNEES/hbrachemi/anaconda/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
hyperparametters = {}
hyperparametters["sigma"]=0.1
hyperparametters["lambda"]=1
hyperparametters["criterion"] = torch.nn.CrossEntropyLoss()
hyperparametters["sponge_optimizer"] = torch.optim.SGD(model_ft.model.parameters(),lr=0.01, momentum=0.9,weight_decay= 5e-4)
hyperparametters["num_sponge_epochs"] = 2
hyperparametters["sponge_criterion"] = "l0"
hyperparametters["num_epochs"] = 50
hyperparametters["criterion"] = torch.nn.CrossEntropyLoss()
hyperparametters["optimizer"] = torch.optim.Adam(model_ft.model.parameters(),lr=0.1)


In [6]:
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

In [7]:
writer = SummaryWriter()

In [8]:
#CLEAN TRAINING

#a = model_ft.train({"train":trainloader,"val":testloader},hyperparametters,writer=writer)
PATH = "../weights_sponge_backdoor/clean/vgg16.pt"

model_ft.model.load_state_dict(torch.load(PATH))

#a=model_ft.evaluate(testloader)

<All keys matched successfully>

In [11]:
print(f"Acc: {a['accuracy']}\n Energy ratio: {np.mean(a['energy']['ratio_cons'])}")

Acc: 0.9099
 Energy ratio: 0.7790136337280273


In [ ]:
#SPONGE TRAINING
#p_ids = [random.randint(1,len(trainset)) for _ in range(int(0.05*len(trainset)))]
pickle_in = open("p_ids_0.05.pickle",'rb')
p_ids = pickle.load(pickle_in)


dataloaders= {"train":trainloader,"val":testloader}

a = model_ft.sponge_train(dataloaders,
                          p_ids,
                          hyperparametters,
                          writer,
                          adaptative_sigma = False,
                          gamma = 1
                         )

model_ft.evaluate(testloader)

/mnt/DONNEES/hbrachemi/anaconda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
